In [1]:
import os
import sys

import openai
from dotenv import load_dotenv, find_dotenv

sys.path.append('../..')

# read local .env file
_ = load_dotenv(find_dotenv())

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']

## Agent 组件入门示例

In [2]:
from langchain.agents import load_tools
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-4")

In [3]:
from langchain_core.exceptions import OutputParserException
import re
from typing import Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain.agents import AgentOutputParser


class RavenOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        print(f"llm_output: {llm_output}")

        if "Action:" in llm_output and "Action Input:" in llm_output:
            action = re.search(r"Action: (.+)", llm_output).group(1)
            action_input = re.search(r"Action Input: (.+)", llm_output).group(1)

            print("Action:", action)
            print("Action Input:", action_input)
            return AgentAction(
                tool=action.strip(),
                tool_input=action_input.strip(),
                log=llm_output
            )
        elif "Final Answer:" in llm_output:
            final_answer = re.search(r"Final Answer: (.+)", llm_output).group(1)
            return AgentFinish(
                return_values={
                    "output": final_answer.strip()
                },
                log=llm_output,
            )
        else:
            raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")


output_parser = RavenOutputParser()

In [4]:
from langchain.agents import ZeroShotAgent, AgentExecutor

tools = load_tools(["serpapi", "llm-math"], llm=llm)
# agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True)  # type: ignore
# agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, output_parser=output_parser,
#                          verbose=True)  # type: ignore
# agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")
# agent = ZeroShotAgent.from_llm_and_tools(llm=llm, tools=tools, output_parser=output_parser, verbose=True)
# agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, output_parser=output_parser, verbose=True)
agent = ZeroShotAgent.from_llm_and_tools(llm=llm, tools=tools, output_parser=output_parser, verbose=True)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [5]:
# agent.run("大连软件园双语学校的位置在哪里？如果检索了多条信息结果，仅回复最新的一条信息。")

In [6]:
# agent.agent.llm_chain.prompt.template
agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

In [7]:
# Agent 组件的数学能力
#agent.run("25 to the power 0.34 is?")
agent_executor.run("25 to the power 0.34 is?")



> Entering new AgentExecutor chain...
llm_output: The question is asking for the value of 25 to the power 0.34.


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `The question is asking for the value of 25 to the power 0.34.

`

In [ ]:
# Agent 组件的使用终端工具能力
agent.run("what files are in my current directory?")

## Agent 组件的应用

In [7]:
# from langchain.chains import LLMChain
# from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
# 
# llm_chain = LLMChain(llm = ChatOpenAI(temperature=0), prompt=prompt)
# tool_names = [tool.name from tool in tools]
# tools = [Tool(name=name, func=func, description=func.__doc__) for name, func in tools.items()]
# agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
# agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
# agent_executor.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")

ValueError: `run` not supported when there is not exactly one output key. Got [].

## FileManagementToolkit 

In [20]:
from tempfile import TemporaryDirectory

from langchain.agents.agent_toolkits import FileManagementToolkit

# We'll make a temporary directory to avoid clutter
working_directory = TemporaryDirectory()

In [21]:
toolkit = FileManagementToolkit(
    root_dir=str(working_directory.name)
)  # If you don't provide a root_dir, operations will default to the current working directory
toolkit.get_tools()

[CopyFileTool(root_dir='D:\\Users\\thinkpad\\AppData\\Local\\Temp\\tmp5e296hfw'),
 DeleteFileTool(root_dir='D:\\Users\\thinkpad\\AppData\\Local\\Temp\\tmp5e296hfw'),
 FileSearchTool(root_dir='D:\\Users\\thinkpad\\AppData\\Local\\Temp\\tmp5e296hfw'),
 MoveFileTool(root_dir='D:\\Users\\thinkpad\\AppData\\Local\\Temp\\tmp5e296hfw'),
 ReadFileTool(root_dir='D:\\Users\\thinkpad\\AppData\\Local\\Temp\\tmp5e296hfw'),
 WriteFileTool(root_dir='D:\\Users\\thinkpad\\AppData\\Local\\Temp\\tmp5e296hfw'),
 ListDirectoryTool(root_dir='D:\\Users\\thinkpad\\AppData\\Local\\Temp\\tmp5e296hfw')]

In [22]:
tools = FileManagementToolkit(
    root_dir=str(working_directory.name),
    selected_tools=["read_file", "write_file", "list_directory"],
).get_tools()
tools

[ReadFileTool(root_dir='D:\\Users\\thinkpad\\AppData\\Local\\Temp\\tmp5e296hfw'),
 WriteFileTool(root_dir='D:\\Users\\thinkpad\\AppData\\Local\\Temp\\tmp5e296hfw'),
 ListDirectoryTool(root_dir='D:\\Users\\thinkpad\\AppData\\Local\\Temp\\tmp5e296hfw')]

In [23]:
read_tool, write_tool, list_tool = tools
write_tool.run({"file_path": "example.txt", "text": "Hello World!"})

'File written successfully to example.txt.'

In [24]:
# List files in the working directory
list_tool.run({})

'example.txt'

In [25]:
read_tool({"file_path": "example.txt"})

'Hello World!'

In [26]:
working_directory.cleanup()

## 自定义 Agent 组件

In [ ]:
from langchain import SerpAPIWrapper

search = SerpAPIWrapper()

